#### Connecting to PostgreSQL

In [1]:
from pathlib import Path

In [2]:
from os import getenv
from dotenv import load_dotenv, find_dotenv

In [3]:
from urllib.parse import quote

In [4]:
load_dotenv()
database_user = "postgres"
database_password = getenv('POSTGRES_PASSWORD')
database_host = "localhost"  # getenv('POSTGRES_HOST')
database_port = 5434
database_name =  "postgres" # getenv('POSTGRES_DB')

In [5]:
postgres_uri = f'postgresql://{database_user}:{quote(database_password)}@{database_host}:{database_port}/{database_name}'

In [6]:
from  psycopg import  connect


In [7]:
database_connection = connect(
    conninfo=postgres_uri,
    autocommit=True,
)

In [8]:
from src.rag.components.shared.databases.postgres import PostgresVectorDBClient

In [9]:
database_client = PostgresVectorDBClient(
    connection=database_connection,
    namespace='my_documents',
)

## Need to create the document table
## Create the chunk table
### write and work on the insertion script.

In [10]:
from src.rag.schemas.document import Document, Node

In [11]:
from psycopg import Connection, sql
from psycopg.pq import TransactionStatus

In [12]:
columns = sql.SQL(", ").join(
    sql.SQL("{column} {data_type}").format(
        column=sql.Identifier(column),
        data_type=sql.SQL(data_type),
    )
    for column, data_type in Document.to_sql_schema().items()
)

In [13]:
columns

Composed([Composed([Identifier('doc_id'), SQL(' '), SQL('TEXT PRIMARY KEY')]), SQL(', '), Composed([Identifier('file_path'), SQL(' '), SQL('TEXT')]), SQL(', '), Composed([Identifier('filename'), SQL(' '), SQL('TEXT')]), SQL(', '), Composed([Identifier('num_pages'), SQL(' '), SQL('INTEGER')]), SQL(', '), Composed([Identifier('coordinate_system'), SQL(' '), SQL('TEXT')]), SQL(', '), Composed([Identifier('table_parsing_kwargs'), SQL(' '), SQL('JSON')]), SQL(', '), Composed([Identifier('last_modified_date'), SQL(' '), SQL('TIMESTAMPTZ')]), SQL(', '), Composed([Identifier('last_accessed_date'), SQL(' '), SQL('TIMESTAMPTZ')]), SQL(', '), Composed([Identifier('creation_date'), SQL(' '), SQL('TIMESTAMPTZ')]), SQL(', '), Composed([Identifier('file_size'), SQL(' '), SQL('INTEGER')]), SQL(', '), Composed([Identifier('object'), SQL(' '), SQL('TEXT')]), SQL(', '), Composed([Identifier('doc_metadata'), SQL(' '), SQL('JSON')])])

In [14]:
database_client.create_table(
    name='documents',
    schema=Document.to_sql_schema(),
    if_not_exists=True,
)

2025-07-29 23:02:51 | INFO     | postgres database client:create_table:108 | Table documents created with schema: {'doc_id': 'TEXT PRIMARY KEY', 'file_path': 'TEXT', 'filename': 'TEXT', 'num_pages': 'INTEGER', 'coordinate_system': 'TEXT', 'table_parsing_kwargs': 'JSON', 'last_modified_date': 'TIMESTAMPTZ', 'last_accessed_date': 'TIMESTAMPTZ', 'creation_date': 'TIMESTAMPTZ', 'file_size': 'INTEGER', 'object': 'TEXT', 'doc_metadata': 'JSON'}


In [15]:
embedding_size = 1024

In [16]:
database_client._full_table_name("documents")

Identifier('my_documents_documents')

In [52]:
database_client.create_table(
    name='nodes',
    schema=Node.to_sql_schema(embedding_dimension=embedding_size, table_prefix="my_documents"),
    if_not_exists=True,
)

2025-07-29 23:44:22 | INFO     | postgres database client:create_table:108 | Table nodes created with schema: {'node_id': 'TEXT PRIMARY KEY', 'variant': 'JSON', 'tokens': 'INTEGER', 'bbox': 'JSON', 'text': 'TEXT', 'elements': 'JSON', 'object': 'TEXT', 'score': 'REAL', 'previous_texts': 'JSON', 'next_texts': 'JSON', 'document_id': 'TEXT', 'embedding': 'vector(1024)'}


In [18]:
from src.rag.components.shared.io import IOManager

In [19]:
document_with_embedding_path = Path.cwd().joinpath(
    "datasets", "parsed_documents_with_embeddings")

In [20]:
io_manager = IOManager(input_document_path=document_with_embedding_path, output_path=document_with_embedding_path)

In [49]:
test_nodes = io_manager.load_nodes_document(start_index=0, end_index=10)

In [50]:
test_nodes_sql = [doc.to_sql_insert(
    "my_documents") for doc in test_nodes]

In [33]:
from uuid import uuid4


In [34]:
document_dict = test_documents[0].document.model_dump()
document_dict["doc_id"] = str(uuid4())  # Ensure doc_id is a string UUID


In [35]:
document_dict

{'file_path': '/Users/esp.py/Documents/Eu-Trip/covid-passport.pdf',
 'filename': 'covid-passport.pdf',
 'num_pages': 1,
 'coordinate_system': 'BOTTOM-LEFT',
 'table_parsing_kwargs': None,
 'last_modified_date': datetime.datetime(2022, 5, 4, 12, 59, 26),
 'last_accessed_date': datetime.datetime(2024, 3, 13, 8, 22, 4, 905879),
 'creation_date': datetime.datetime(2023, 6, 20, 13, 12, 6, 511693),
 'file_size': 321769,
 'object': 'ingest.document',
 'doc_metadata': None,
 'doc_id': '3c3780bc-a40c-43a9-a2db-4d370bab1fb0'}

In [41]:
document = database_client.find_by_id_or_create("documents", document_dict, id_field="doc_id")

2025-07-29 23:15:36 | INFO     | postgres database client:find_by_id_or_create:206 | Executing query: SELECT * FROM "my_documents_documents" WHERE "doc_id" = %s


In [42]:
document

(False,
 [('3c3780bc-a40c-43a9-a2db-4d370bab1fb0',
   '/Users/esp.py/Documents/Eu-Trip/covid-passport.pdf',
   'covid-passport.pdf',
   1,
   'BOTTOM-LEFT',
   None,
   datetime.datetime(2022, 5, 4, 12, 59, 26, tzinfo=zoneinfo.ZoneInfo(key='Etc/UTC')),
   datetime.datetime(2024, 3, 13, 8, 22, 4, 905879, tzinfo=zoneinfo.ZoneInfo(key='Etc/UTC')),
   datetime.datetime(2023, 6, 20, 13, 12, 6, 511693, tzinfo=zoneinfo.ZoneInfo(key='Etc/UTC')),
   321769,
   'ingest.document',
   None)])

For insertion: 

- This guide recomend: 
    - remove indexes
    - remove foreign keys and then insert data and add them back once the data is insert.
    https://www.postgresql.org/docs/current/populate.html

data

In [81]:
data_insert = database_client.bulk_insert(
    table_name="nodes",
    data=test_nodes_sql,
    returning=None)

2025-07-30 00:09:04 | INFO     | postgres database client:bulk_insert:286 | Executing bulk insert query: INSERT INTO "my_documents_nodes" ("node_id", "variant", "tokens", "bbox", "text", "elements", "object", "score", "previous_texts", "next_texts", "document_id", "embedding")  values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)  ON CONFLICT DO NOTHING


In [82]:
data_insert

In [58]:
len(columns)

12

In [69]:
placeholders=sql.SQL(", ").join(sql.Placeholder() * len(columns)
)

In [71]:
values = [tuple(item[col] for col in columns) for item in test_nodes_sql]

In [73]:
len(values[0])

12

In [ ]:
# TODO: Done with the insertion script, tommorow come back and check

Write everything in one script, 
add the constraints, such as primary key and foreign key, and indexes